In [33]:
import os
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shutil
import cv2
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import random
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, MaxPooling2D, Flatten, Dropout, Conv2D, GlobalAveragePooling2D
from tensorflow.keras.applications import VGG16

In [14]:
DATAPATH = 'D:/DATASETS/datasets/FaceMaskDataset/'
MASKPATH = 'D:/DATASETS/datasets/FaceMaskDataset/with_mask/'
NOMASKPATH = 'D:/DATASETS/datasets/FaceMaskDataset/without_mask/'
TESTPATH = 'D:/DATASETS/datasets/FaceMaskDataset/test'

In [15]:
def view(pth):
    images=list()
    for img in random.sample(os.listdir(pth),9):
        images.append(img)
        i = 0
        fig,ax = plt.subplots(nrows=3, ncols=3, figsize=(30,20))
    for row in range(3):
        for col in range(3):
            ax[row,col].imshow(cv2.imread(os.path.join(pth,images[i])))
            i+=1

In [16]:
len(os.listdir(MASKPATH)),len(os.listdir(NOMASKPATH))

(1897, 1918)

In [17]:
BATCH_SIZE=16

In [18]:
trainGen = ImageDataGenerator(
rescale=1/255.,
horizontal_flip=True,
validation_split = 0.1
)
testGen =ImageDataGenerator(
rescale= 1/255.,
)

In [19]:
train = trainGen.flow_from_directory(
    DATAPATH, 
    target_size=(224, 224),
    classes=['with_mask','without_mask'],
    class_mode='categorical', 
    batch_size=BATCH_SIZE, 
    shuffle=True,
    subset='training'
)

validation = trainGen.flow_from_directory(
    DATAPATH, 
    target_size=(224, 224),
    classes=['with_mask','without_mask'],
    class_mode='categorical', 
    batch_size=BATCH_SIZE, 
    shuffle=True,
    subset='validation'
)

Found 3435 images belonging to 2 classes.
Found 380 images belonging to 2 classes.


In [20]:
IMG_WIDTH = 224 
IMG_HEIGHT = 224
IMG_DEPTH = 3

In [21]:
conv_base = VGG16(weights='C:/Users/Drishya/Downloads/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',
                  include_top=False, 
                  input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_DEPTH)
                 )
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [22]:
maskvgg = Sequential()
maskvgg.add(conv_base)
maskvgg.add(GlobalAveragePooling2D())
maskvgg.add(Dense(64,activation='relu'))
maskvgg.add(Dropout(0.3))
maskvgg.add(Dense(2,activation='softmax'))
maskvgg.summary()
maskvgg.compile(optimizer=Adam(),loss='categorical_crossentropy',metrics=['acc'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                32832     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 130       
Total params: 14,747,650
Trainable params: 14,747,650
Non-trainable params: 0
_________________________________________________________________


In [25]:
checkpoint = ModelCheckpoint(
    'modelvgg.h5',
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode='min'
)

In [26]:
hist = maskvgg.fit(
    train,
    epochs = 20,
    validation_data = validation,
    callbacks = [checkpoint]
)

Epoch 1/20
 69/215 [========>.....................] - ETA: 37:09 - loss: 0.7753 - acc: 0.4882

D:\Anaconda3\lib\site-packages\PIL\Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


215/215 [==============================] - ETA: 0s - loss: 0.7348 - acc: 0.5039 
Epoch 00001: val_loss improved from inf to 0.69847, saving model to modelvgg.h5
215/215 [==============================] - 2332s 11s/step - loss: 0.7348 - acc: 0.5039 - val_loss: 0.6985 - val_acc: 0.5026
Epoch 2/20
215/215 [==============================] - ETA: 0s - loss: 0.6789 - acc: 0.5639
Epoch 00002: val_loss improved from 0.69847 to 0.40950, saving model to modelvgg.h5
215/215 [==============================] - 1795s 8s/step - loss: 0.6789 - acc: 0.5639 - val_loss: 0.4095 - val_acc: 0.8316
Epoch 3/20
215/215 [==============================] - ETA: 0s - loss: 0.7432 - acc: 0.5668
Epoch 00003: val_loss did not improve from 0.40950
215/215 [==============================] - 1829s 9s/step - loss: 0.7432 - acc: 0.5668 - val_loss: 0.6938 - val_acc: 0.4842
Epoch 4/20
215/215 [==============================] - ETA: 0s - loss: 0.7079 - acc: 0.5560
Epoch 00004: val_loss did not improve from 0.40950
215/215 [=

In [27]:
maskvgg.evaluate(validation)

24/24 [==============================] - 49s 2s/step - loss: 0.6932 - acc: 0.4974


[0.6932325959205627, 0.4973684251308441]

In [31]:
maskvgg.save('modelvgg.h5')

In [32]:
from tensorflow.keras.models import load_model
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import Conv2D,MaxPooling2D,SpatialDropout2D,Flatten,Dropout,Dense
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
import cv2
import datetime

from tensorflow.keras.preprocessing import image

mymodel = load_model('modelvgg.h5')

cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier('D:/haarcascade_frontalface_default.xml')

while cap.isOpened():
    _,img=cap.read()
    face=face_cascade.detectMultiScale(img,scaleFactor=1.1,minNeighbors=4)
    for(x,y,w,h) in face:
        face_img = img[y:y+h, x:x+w]
        cv2.imwrite('temp.jpg',face_img)
        test_image=image.load_img('temp.jpg',target_size=(150,150,3))
        test_image=image.img_to_array(test_image)
        test_image=np.expand_dims(test_image,axis=0)
        pred=mymodel.predict(test_image)[0][0]
        if pred==1:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),3)
            cv2.putText(img,'NO MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3)
        else:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),3)
            cv2.putText(img,'MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),3)
        datet=str(datetime.datetime.now())
        cv2.putText(img,datet,(400,450),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)
          
    cv2.imshow('img',img)
    
    if cv2.waitKey(1)==ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()
import cv2

mymodel=load_model('modelvgg.h5')

cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier('D:/haarcascade_frontalface_default.xml')

while cap.isOpened():
    _,img=cap.read()
    face=face_cascade.detectMultiScale(img,scaleFactor=1.1,minNeighbors=4)
    for(x,y,w,h) in face:
        face_img = img[y:y+h, x:x+w]
        cv2.imwrite('temp.jpg',face_img)
        test_image=image.load_img('temp.jpg',target_size=(150,150,3))
        test_image=image.img_to_array(test_image)
        test_image=np.expand_dims(test_image,axis=0)
        pred=mymodel.predict(test_image)[0][0]
        if pred==1:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),3)
            cv2.putText(img,'NO MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3)
        else:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),3)
            cv2.putText(img,'MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),3)
        datet=str(datetime.datetime.now())
        cv2.putText(img,datet,(400,450),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)
          
    cv2.imshow('img',img)
    
    if cv2.waitKey(1)==ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

In [34]:
trainGen = ImageDataGenerator(
rescale=1/255.,
horizontal_flip=True,
validation_split = 0.1
)
testGen =ImageDataGenerator(
rescale= 1/255.,
)
train = trainGen.flow_from_directory(
    'D:/DATASETS/datasets/FaceMaskDataset/train/', 
    target_size=(224, 224),
    classes=['with_mask','without_mask'],
    class_mode='categorical', 
    batch_size=BATCH_SIZE, 
    shuffle=True,
    subset='training'
)

validation = trainGen.flow_from_directory(
    'D:/DATASETS/datasets/FaceMaskDataset/test/', 
    target_size=(224, 224),
    classes=['with_mask','without_mask'],
    class_mode='categorical', 
    batch_size=BATCH_SIZE, 
    shuffle=True,
    subset='validation'
)

IMG_WIDTH = 224 
IMG_HEIGHT = 224
IMG_DEPTH = 3

conv_base = VGG16(weights='C:/Users/Drishya/Downloads/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',
                  include_top=False, 
                  input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_DEPTH)
                 )
conv_base.summary()

maskvgg = Sequential()
maskvgg.add(conv_base)
maskvgg.add(GlobalAveragePooling2D())
maskvgg.add(Dense(64,activation='relu'))
maskvgg.add(Dropout(0.3))
maskvgg.add(Dense(2,activation='softmax'))
maskvgg.summary()
maskvgg.compile(optimizer=Adam(),loss='categorical_crossentropy',metrics=['acc'])

checkpoint = ModelCheckpoint(
    'modellvgg.h5',
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode='min'
)

hist = maskvgg.fit(
    train,
    epochs = 15,
    validation_data = validation,
    callbacks = [checkpoint]
)
maskvgg.save('modell_vgg.h5')

Found 2268 images belonging to 2 classes.
Found 128 images belonging to 2 classes.
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
____________________________________________

In [35]:
maskvgg.evaluate(validation,verbose=0)

[0.8780681490898132, 0.671875]

In [38]:
#IMPLEMENTING LIVE DETECTION OF FACE MASK
from tensorflow.keras.models import load_model
import numpy as np
from tensorflow import keras
import tensorflow.keras.backend as k
from tensorflow.keras.layers import Conv2D,MaxPooling2D,SpatialDropout2D,Flatten,Dropout,Dense
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
import cv2
import datetime

from tensorflow.keras.preprocessing import image
import cv2
mymodel = load_model('modell_vgg.h5')

cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier('D:/haarcascade_frontalface_default.xml')

while cap.isOpened():
    _,img=cap.read()
    face=face_cascade.detectMultiScale(img,scaleFactor=1.1,minNeighbors=4)
    for(x,y,w,h) in face:
        face_img = img[y:y+h, x:x+w]
        cv2.imwrite('temp.jpg',face_img)
        test_image=image.load_img('temp.jpg',target_size=(150,150,3))
        test_image=image.img_to_array(test_image)
        test_image=np.expand_dims(test_image,axis=0)
        pred=mymodel.predict(test_image)[0][0]
        if pred==1:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),3)
            cv2.putText(img,'NO MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3)
        else:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),3)
            cv2.putText(img,'MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),3)
        datet=str(datetime.datetime.now())
        cv2.putText(img,datet,(400,450),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)
          
    cv2.imshow('img',img)
    
    if cv2.waitKey(1)==ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()